In [6]:




import datetime
import pandas as pd
import os
import folium

# Get current date and time
now = datetime.datetime.now()


year = now.strftime("%Y")
month = now.strftime("%m")
day = now.strftime("%d")

dir_path = f'./data/real_time/{year}/{month}/{day}/'

# Get a list of all the csv files in the target directory

files = [f for f in os.listdir(dir_path) if f.endswith('.csv')]

# Initialize an empty dataframe
combined_df = pd.DataFrame()

# Loop through the files and read each into a dataframe
for file in files:
    file_path = os.path.join(dir_path, file)
    temp_df = pd.read_csv(file_path)

    # Append each dataframe to the combined_df
    combined_df = combined_df.append(temp_df, ignore_index=True)


/var/folders/8n/h7fgcf_96jbfwn6vj1l07p3h0000gn/T/ipykernel_10897/3898257807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(temp_df, ignore_index=True)
/var/folders/8n/h7fgcf_96jbfwn6vj1l07p3h0000gn/T/ipykernel_10897/3898257807.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(temp_df, ignore_index=True)


In [7]:
combined_df

,date_EST,status,aircraft.iataCode,aircraft.icao24,aircraft.icaoCode,aircraft.regNumber,airline.iataCode,airline.icaoCode,nameAirline,flight.iataNumber,...,arrival.airportId,arrival.nameAirport,arrival.iataCode,arrival.icaoCode,arrival.codeIataCity,arrival.codeIso2Country,arrival.nameCountry,arrival.latitudeAirport,arrival.longitudeAirport,arrival.timezone
0,2023-05-18 15:47:18,landed,B752,A8DF06,B752,N6707A,DL,DAL,Delta Air Lines,DL447,...,7224.0,John Wayne,SNA,KSNA,SNA,US,United States,33.680202,-117.860535,America/Los_Angeles
1,2023-05-18 15:47:18,en-route,B753,A77AE0,B753,N581NW,DL,DAL,Delta Air Lines,DL895,...,4189.0,Los Angeles International,LAX,KLAX,LAX,US,United States,33.943398,-118.408280,America/Los_Angeles
2,2023-05-18 15:47:18,en-route,B752,A8DE56,B752,N6702,DL,DAL,Delta Air Lines,DL1375,...,2344.0,Fort Lauderdale–Hollywood International,FLL,KFLL,FLL,US,United States,26.071491,-80.144905,America/New_York
3,2023-05-18 15:47:18,en-route,A321,A41F37,A321,N365DN,DL,DAL,Delta Air Lines,DL1173,...,5966.0,Palm Beach International Airport,PBI,KPBI,PBI,US,United States,26.688885,-80.090440,America/New_York
4,2023-05-18 15:47:18,en-route,A321,A44308,A321,N374DX,DL,DAL,Delta Air Lines,DL423,...,7174.0,Salt Lake City International,SLC,KSLC,SLC,US,United States,40.785645,-111.980675,America/Denver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,2023-05-18 15:24:12,en-route,B77W,06A078,B77W,A7-BAK,QR,QTR,Qatar Airways,QR755,...,411.0,Hartsfield-jackson Atlanta International,ATL,KATL,ATL,US,United States,33.640068,-84.444030,America/New_York
540,2023-05-18 15:24:12,en-route,E55P,A390D5,E55P,N329MC,NaN,NaN,NaN,XXD,...,411.0,Hartsfield-jackson Atlanta International,ATL,KATL,ATL,US,United States,33.640068,-84.444030,America/New_York
541,2023-05-18 15:24:12,en-route,B763,A49CE2,B763,N397AZ,NaN,NaN,NaN,8C3434,...,411.0,Hartsfield-jackson Atlanta International,ATL,KATL,ATL,US,United States,33.640068,-84.444030,America/New_York
542,2023-05-18 15:24:12,en-route,A320,A81B2A,A320,N621NK,NK,NKS,Spirit Airlines,NK1784,...,411.0,Hartsfield-jackson Atlanta International,ATL,KATL,ATL,US,United States,33.640068,-84.444030,America/New_York


In [8]:
# create the map
m = folium.Map(location=[0, 0], zoom_start=2)

# group by 'aircraft.icao24' to gather all points related to the same flight
grouped = combined_df.groupby('aircraft.icao24')

for name, group in grouped:
    latitudes = group['geography.latitude']
    longitudes = group['geography.longitude']

    # Create a line for the entire flight path
    folium.PolyLine(locations=list(zip(latitudes, longitudes)), color="blue", weight=2.5, opacity=1).add_to(m)

    # Optional: Adding markers for the start and end of each flight
    folium.Marker([latitudes.iloc[0], longitudes.iloc[0]], popup=f'Start of Flight {name}').add_to(m)
    folium.Marker([latitudes.iloc[-1], longitudes.iloc[-1]], popup=f'End of Flight {name}').add_to(m)

# Display the map
m